# Формирование электроннорй базы данных противопожарного водоснабжения

## Принятые сокращения

* **БД ИНППВ** - База данных источников наружного противопожарного водоснабжения

## Выбор способа реализации базы данных

## Выбор способа оцифровывания

Исходными данными для формирования БД ВИ являются:
- Справочник улиц водоисточников  Центрального района 10 ПСО 26 ПСЧ ФПС ГУ МЧС России по Иркутской области (Далее - Справочник водоисточников);
- Планшет водоисточников Центрального округа г. Братска (Далее - Планшет водоисточников)

Два этих документа дополняют друг друга. Планшет водоисточников содержит информацию о пространственном положении ВИ на местности в районе выезда ПСЧ-26. Справочник же содержит информацию описывающую характеристики ВИ имеющие значение при тушении пожаров и проведении проверок наружного противопожарного водоснабжения.

Справочник улиц представляет собой таблицу состоящую из 11 столбцов и 631 строки. Каждая строка содержит запись данных об одном источнике противопожарного водоснабжения. Полями записи являются колонки таблицы (рис.1):

1. № п/п
2. Наименование улиц
3. № части, района выезда
4. Вид и номер водоисточника
5. Расположение гидранта	
6. Место расположения указателя	
7. Координаты согласно указателя	
8. Характеристика сети и  напор	
9. Водоотдача сети, л (объем)	
10. Неисправности пожарных гидрантов	
11. Ведомственная принадлежность 

![Фрагмент таблицы Справочника улиц](src/inppwTable_example.jpg)

Рисунок 1 - Фрагмент таблицы Справочника улиц

Планшет водоисточников представляет собой графическое представление фрагментов карты района выезда ПСЧ-26 с нанесенными отметками ВИ и расстояниями до них, позволяющими определить точное местоположение водоисточников при проведении боевых действий по тушению пожаров (рис. 2).

![Пример карты района выезда из планшета водоисточников](src/planshet_example.jpg)

Рисунок 2 - Пример карты района выезда из планшета водоисточников

По изображениям 1 и 2 видно, что ни тот ни другой источник не содержит данных о географических координатах водоисточников, а значит точное их позиционирование возможно только при ручном указании их пространственного положения. Процесс расстановки 631 ВИ на карте "в ручную", даже с использованием современных средств вычислительной техники представляется длительным и трудоемким, сопряженным с вероятностью возникновения неточностей связанных с человеческим фактором. Поэтому следует попытаться найти способ упростить данный процесс.

Поскольку в справочнике водоисточников имеются точные сведения о адресе здания к которому привязан ВИ, то предлагается использовать географические координаты зданий в качестве чернового варианта расположения ВИ, который можно определить в автоматическом режиме. Когда же эти координаты будут получены, можно будет уточнить окончательное расположение ВИ в соответствии с отметками в планшете водоисточников. Такой подход должен позволить сократить временные затраты на поиск точных географических координат ВИ. Кроме того, он позволит избежать необходимости в ручную вносить атрибутивные данные о ВИ из справочника водоисточников.

Географические координаты зданий и сооружений г. Братска были предоставлены отделом информационных технологий и компьютерного моделирования научно-технического центра ФГБОУ ВО Сибирская пожарно-спасательная академия ГПС МЧС России.

Для обработки данных будет использован язык программирования Python и следующие программные библиотеки:
- pandas - библиотека для работы с наборами данных в формате таблиц
- nltk - набор инструментов для анализа естественного языка
- geopandas - расширение библиотеки pandas для работы с географически ориентированными объектами

Визуализация данных о ИНППВ будет осуществляться посредством свободного программного обеспечения QGIS.

Таким образом окончательный порядок оцифровывания сведений о ВИ будет выглядеть следующим образом:

1. Формирование таблицы с данными о ВИ в формате электронной таблицы Excel
2. Загрузка данных в виде набора данных pandas
3. Проверка и очистка данных (удаление не нужных знаков препинания, префиксов, неинформативных частей речи)
4. Поиск географических координат зданий к которым привязаны ВИ
5. Экспорт полученных данных в формат geopackage
6. Добавление данных на карту QGIS
7. Уточнение точного положения ВИ на карте QGIS
8. Размещение на карте QGIS ВИ для которых определить координаты не удалось

## 1. Формирование таблицы с данными о ВИ в формате электронной таблицы Excel

Данные из таблицы документа Word были перенесены в электронную таблицу Excel (Рис. 3)

![Данные о ВИ в формате Excel](src/inppwTable_excel.jpg)

Рисунок 3 - Данные о ВИ в формате Excel

Данные были перенесены "дословно", без изменений.

## 2. Загрузка данных в виде набора данных pandas

Для эффективной дальнейшей работы загрузим данные о ВИ из полученного файла Excel в формате набора данных pandas

In [14]:
# подключение библиотеки pandas
import pandas as pd

# Загрузка данных о ВИ
water_sources = pd.read_excel("Справочник ВИ.xls")
# Временно переименуем колонки для простоты работы
water_sources.columns = ['id', 'адрес', 'ПСЧ', 'номер', 'расположение', 'привязка', 'направление', 'тип сети', 'водоотдача', 'неисправности', 'владелец']
# Установим индекс строк
water_sources.index = water_sources['id']
# Выведем первые 5 записей
water_sources.head()

,id,адрес,ПСЧ,номер,расположение,привязка,направление,тип сети,водоотдача,неисправности,владелец
id,,,,,,,,,,,
1,1,ул. Южная д. №1,26 ПСЧ,ПГ-1,д.1,дом,0-4-13,Н-40 К-250,185,-,МП «ДГИ» МО г.Братска
2,2,ул. Южная д. №7,26 ПСЧ,ПГ-2,д.7,дом,0-3-13,Н-40 К-250,185,-,МП «ДГИ» МО г.Братска
3,3,ул. Южная д. №17,26 ПСЧ,ПГ-3,д.17,дом,0-4-14,Н-40 К-250,185,-,МП «ДГИ» МО г.Братска
4,4,ул. Южная д. № 21,26 ПСЧ,ПГ-4,д.21,дом,0-11-19,Н-40 К-250,185,-,МП «ДГИ» МО г.Братска
5,5,ул. Южная д. № 23,26 ПСЧ,ПГ-5,д.23,дом,0-10-17,Н-40 К-250,185,-,МП «ДГИ» МО г.Братска


## 3. Проверка и очистка данных

На следующем этапе следует провести подготовку данных к их интерпретации с использованием языка программирования Python и инструментов анализа натурального языка (nltk).

Так в строке 1 (рис. 3) в столбце содержится следующая запись:

    ул. Южная д. №1 

В это строке имеются некоторые не представляющие ценности данные. Например префиксы "ул.", "д.", символ "№", а так же знаки препинания. Их следует удалить, прежде чем передавать адрес для анализа.

Следует удалить столбец "Расположение гидранта", так как сведения о номере дома уже содержатся в столбце "Наименование улиц", который следует переименовать в "Адрес"

Наконец, очевидно, что сведения о напоре в пожарном гидранте в столбце "Характеристика сети и  напор" не представляют ценности и их следует удалить, так как для всех записей в таблице они равны "Н-40".

In [15]:
# Удалим столбец "расположение"
water_sources = water_sources[['id', 'адрес', 'ПСЧ', 'номер', 'привязка', 'направление', 'тип сети', 'водоотдача', 'неисправности', 'владелец']]

In [16]:
# Откорректируем сведения в наборе данных
# Перебираем все записи
for key, ws in water_sources.iterrows():
    # корректируем записи о адресе
    address = ws['адрес']
    
    address = address.replace('№ ', ' ')
    address = address.replace('№', ' ')    
    address = address.replace('ул. ', '')
    address = address.replace('ул.', '')
    address = address.replace('д. ', '')
    address = address.replace('д.', '')
    address = address.replace('бул.', 'бульвар')
    address = address.replace('пер.', 'переулок')
    address = address.replace('  ', ' ')
    address = address.strip()

    water_sources.loc[key, 'адрес'] = address

    # корректируем записи о типе сети
    pipe = ws['тип сети']
    pipe = pipe.replace('Н-40 ', '')
    pipe = pipe.replace('Н-40', '')

    water_sources.loc[key, 'тип сети'] = pipe

# Выведем первые 10 записей
water_sources.head()

,id,адрес,ПСЧ,номер,привязка,направление,тип сети,водоотдача,неисправности,владелец
id,,,,,,,,,,
1,1,Южная 1,26 ПСЧ,ПГ-1,дом,0-4-13,К-250,185,-,МП «ДГИ» МО г.Братска
2,2,Южная 7,26 ПСЧ,ПГ-2,дом,0-3-13,К-250,185,-,МП «ДГИ» МО г.Братска
3,3,Южная 17,26 ПСЧ,ПГ-3,дом,0-4-14,К-250,185,-,МП «ДГИ» МО г.Братска
4,4,Южная 21,26 ПСЧ,ПГ-4,дом,0-11-19,К-250,185,-,МП «ДГИ» МО г.Братска
5,5,Южная 23,26 ПСЧ,ПГ-5,дом,0-10-17,К-250,185,-,МП «ДГИ» МО г.Братска


## 4. Поиск географических координат зданий к которым привязаны ВИ

Для начала следует загрузить данные о зданиях г.Братска

In [7]:
# Сформируем набор записей о зданиях
buildings = pd.read_excel("Bratsk_buildings.xlsx", sheet_name='Дома')
# Оставим только записи относящиеся к г. Братск
print("Количество записей до фильтрации - {}".format(len(buildings)))
buildings = buildings[buildings['city']=='Братск']
print("Количество записей после фильтрации - {}".format(len(buildings)))
# Выведем первые 5 записей
buildings.head()

Количество записей до фильтрации - 18796
Количество записей после фильтрации - 15611


,street_1,number_1,city,purpose,name,post_index,geo_lon,geo_lat
11,Курчатова,44,Братск,"Детский сад, ясли",NaN,665726.0,101.631009,56.160976
12,Обручева,23а,Братск,"Детский сад, ясли",NaN,665726.0,101.630730,56.159747
13,Курчатова,60,Братск,"Детский сад, ясли",NaN,665726.0,101.636236,56.159874
14,Кирова,31а,Братск,"Детский сад, ясли",NaN,665708.0,101.630288,56.154590
15,Советская,1а,Братск,"Детский сад, ясли",NaN,665729.0,101.638112,56.151803


In [8]:
# Подготовим набор для работы:
buildings['building'] = buildings['street_1'] + ' ' + buildings['number_1']
buildings = buildings[['building', 'geo_lon', 'geo_lat']]
# Устанавливаем в качестве индекса адрес здания для быстрого поиска точного совпадения
buildings.index = buildings['building']
# Выведем первые 5 записей
buildings.head()

,building,geo_lon,geo_lat
building,,,
Курчатова 44,Курчатова 44,101.631009,56.160976
Обручева 23а,Обручева 23а,101.630730,56.159747
Курчатова 60,Курчатова 60,101.636236,56.159874
Кирова 31а,Кирова 31а,101.630288,56.154590
Советская 1а,Советская 1а,101.638112,56.151803


In [19]:
# Собственно анализ
# Загружаем библиотеку nltk
import nltk
import time

In [ ]:
cur_time = time.time() # Засекаем текущее время
i=0
# Минимальная необходимая точность совпадения записи
accuracy_rate=0.3
# Перебираем все записи адресов ВИ
for key, ws in water_sources.iterrows():
    item_time = time.time() # текущее время данной записи
    
    address = ws['адрес']
    coords=[]
    try:
        # Проверяем наличие точных совпадений
        building = buildings.loc[address]
        # Если не вызвана ошибка, значит точное совпадение найдено и можно указать координаты записи о здании
        water_sources.loc[key,'geo_lon']=building['geo_lon']
        water_sources.loc[key,'geo_lat']=building['geo_lat']
        water_sources.loc[key,'need_check']=0

        print("{} '{}', точное совпадение, затрачено {} с".format(i, address, round(time.time() - item_time, 2)))
    except:
        # Если точного совпадения нет, то ищем его  использованием nltk
        minDiff = 1.0
        minAddress = ''
        for key_gis, rec_gis in buildings.iterrows():
            buildings_gis_address = rec_gis['building']
            sovp = nltk.edit_distance(address, buildings_gis_address)/len(address)   #Ищем минимальное редакторское расстояние
            if (sovp < minDiff):      
                minDiff=sovp 
                minAddress=buildings_gis_address
                coords=[rec_gis['geo_lon'], rec_gis['geo_lat']]

        # Если различия достаточно невелики, сохраняем их в капчествен наиболее вероятного варианта
        if minDiff<accuracy_rate:
            print("{} Было '{}', стало '{}', с минимальной разностью {}, затрачено {} с".format(i, address, minAddress, minDiff, round(time.time() - item_time, 2)))
            water_sources.loc[key,'geo_lon']=coords[0]
            water_sources.loc[key,'geo_lat']=coords[1]
        else:
            print('{} Подходящее значение не найдено: {}, затрачено {} с'.format(i, address, round(time.time() - item_time, 2)))

        water_sources.loc[key,'need_check']=1

    i+=1

print('Время обработки: ', round(time.time() - cur_time, 2), 'c', sep='')

In [1]:
# import pandas as pd
# water_sources = pd.read_csv('Bratsk_inppw.csv')

In [3]:
# Напечатаем первые 10 записей
water_sources.head(10)

,id,id.1,адрес,ПСЧ,номер,привязка,направление,тип сети,водоотдача,неисправности,владелец,geo_lon,geo_lat,need_check
0,1,1,Южная 1,26 ПСЧ,ПГ-1,дом,0-4-13,К-250,185,-,МП «ДГИ» МО г.Братска,101.610997,56.148392,1.0
1,2,2,Южная 7,26 ПСЧ,ПГ-2,дом,0-3-13,К-250,185,-,МП «ДГИ» МО г.Братска,101.626775,56.143269,1.0
2,3,3,Южная 17,26 ПСЧ,ПГ-3,дом,0-4-14,К-250,185,-,МП «ДГИ» МО г.Братска,101.610997,56.148392,1.0
3,4,4,Южная 21,26 ПСЧ,ПГ-4,дом,0-11-19,К-250,185,-,МП «ДГИ» МО г.Братска,101.612736,56.147960,1.0
4,5,5,Южная 23,26 ПСЧ,ПГ-5,дом,0-10-17,К-250,185,-,МП «ДГИ» МО г.Братска,101.612736,56.147960,0.0
5,6,6,Южная 23,26 ПСЧ,ПГ-6,дом,0-9-14,К-250,185,-,МП «ДГИ» МО г.Братска,101.612736,56.147960,0.0
6,7,7,Мира 1,26 ПСЧ,ПГ-7,дом,0-2-21,К-250,185,-,МП «ДГИ» МО г.Братска,101.614693,56.147759,0.0
7,8,8,Мира 5,26 ПСЧ,ПГ-8,дом,0-9-21,К-250,185,-,МП «ДГИ» МО г.Братска,101.615843,56.148927,0.0
8,9,9,Мира 1-Южная,26 ПСЧ,ПГ-9,дом,0-10-14,К-250,185,-,МП «ДГИ» МО г.Братска,NaN,NaN,1.0
9,10,10,Мира 17,26 ПСЧ,ПГ-10,дом,0-7-21,К-250,185,-,МП «ДГИ» МО г.Братска,101.616755,56.149852,0.0


In [4]:
print("Координаты не найдены для {} записей".format(len(water_sources[water_sources['geo_lon'].isna()==True])))
print("Требуется уточнение привязки к адресу для {} записей".format(len(water_sources[water_sources['need_check']==1])))
print("Точный адрес найден для {} записей".format(len(water_sources[water_sources['need_check']==0])))

Координаты не найдены для 116 записей
Требуется уточнение привязки к адресу для 256 записей
Точный адрес найден для 375 записей


## 5. Экспорт полученных данных в формат geopackage

Далее преобразуем полученные данные в формат geopandas для последующего сохранения в виде пространственно ориентированного набора данных geopackage.

In [7]:
# Загрузим библиотеку geopandas
import geopandas as gpd

# Сформируем новый набор географически привязанных данных
gdf = gpd.GeoDataFrame(
    water_sources, geometry=gpd.points_from_xy(water_sources.geo_lon, water_sources.geo_lat), crs='WGS 84')
# Выведем первые 5 записей
gdf.head()

,id,id.1,адрес,ПСЧ,номер,привязка,направление,тип сети,водоотдача,неисправности,владелец,geo_lon,geo_lat,need_check,geometry
0,1,1,Южная 1,26 ПСЧ,ПГ-1,дом,0-4-13,К-250,185,-,МП «ДГИ» МО г.Братска,101.610997,56.148392,1.0,POINT (101.61100 56.14839)
1,2,2,Южная 7,26 ПСЧ,ПГ-2,дом,0-3-13,К-250,185,-,МП «ДГИ» МО г.Братска,101.626775,56.143269,1.0,POINT (101.62677 56.14327)
2,3,3,Южная 17,26 ПСЧ,ПГ-3,дом,0-4-14,К-250,185,-,МП «ДГИ» МО г.Братска,101.610997,56.148392,1.0,POINT (101.61100 56.14839)
3,4,4,Южная 21,26 ПСЧ,ПГ-4,дом,0-11-19,К-250,185,-,МП «ДГИ» МО г.Братска,101.612736,56.147960,1.0,POINT (101.61274 56.14796)
4,5,5,Южная 23,26 ПСЧ,ПГ-5,дом,0-10-17,К-250,185,-,МП «ДГИ» МО г.Братска,101.612736,56.147960,0.0,POINT (101.61274 56.14796)


In [8]:
# Сохраним набор данных в файл geopackage ws_bratsk.gpkg
gdf.to_file('ws_bratsk.gpkg', driver='GPKG', layer='FRU-64')

## 6. Добавление данных на карту QGIS

Создадим новый картографический проект QGIS для города Братск и добавим в него слой водоисточников из полученного ранее файла  файла fires_ZLC.gpkg. В качестве картографической подложки будем использовать карту OpenStreetMap (рис. 4)

![Необработанные метки ВИ на карте Братска](src/inppwQGIS_0.jpg)

Рисунок 4 - Необработанные метки ВИ на карте Братска

Далее следует уточнить пространственное положение ВИ. Для этого на схеме выделим цветом метки ВИ с точно определенными координатами зданий и в качестве подписи выведем адрес ВИ (рис. 5)

![Необработанные метки ВИ на карте Братска](src/inppwQGIS_1.jpg)

Рисунок 5 - Необработанные метки ВИ на карте Братска

Точки с координатами определенными примерно следует сверить с реальным положением зданий с соотвествующими адресами. Точки для которых координаты определены точно, достаточно сверить по местоположению относительно зданий.

## 7. Уточнение точного положения ВИ на карте QGIS

Далее сверяя положение меток ВИ полученных в результате интерпретации адреса здания к которому он привязан с метками в планшете водоисточников. Сверка осуществляется по листам. После сверки каждого листа планшета водоисточников метки на карте посредством таблицы атрибутов указываются как проверенные - для столбца "need_check" указывается значение 2 (рис. 6).

![Метки ВИ отмеченные как "проверенные" и таблица атрибутов слоя ВИ](src/inppwQGIS_2.jpg)

Рисунок 6 - Метки ВИ отмеченные как "проверенные" и таблица атрибутов слоя ВИ


## 8. Размещение на карте QGIS ВИ для которых определить координаты не удалось

In [5]:
water_sources_na = water_sources[water_sources['geo_lon'].isna()==True]
print("количество нераспознанных записей {}".format(len(water_sources_na)))
water_sources_na.head()

количество нераспознанных записей 116


,id,id.1,адрес,ПСЧ,номер,привязка,направление,тип сети,водоотдача,неисправности,владелец,geo_lon,geo_lat,need_check
8,9,9,Мира 1-Южная,26 ПСЧ,ПГ-9,дом,0-10-14,К-250,185,-,МП «ДГИ» МО г.Братска,NaN,NaN,1.0
34,35,35,Мира 33 – Кирова,26 ПСЧ,ПГ-7,дом,0-22-24,К-250,185,-,МП «ДГИ» МО г.Братска,NaN,NaN,1.0
128,129,129,переулок Новый 4,26 ПСЧ,ПГ-45,дом,0-17-0,К-250,235,-,МП «ДГИ» МО г.Братска,NaN,NaN,1.0
129,130,130,переулок Новый 6 а,26 ПСЧ,ПГ-46,дом,0-1-17,К-250,235,-,МП «ДГИ» МО г.Братска,NaN,NaN,1.0
191,192,192,ДМБ,26 ПСЧ,ПГ-9,Хоз. блок,0-19-46,К-250,185,-,МП «ДГИ» МО г.Братска,NaN,NaN,1.0


In [6]:
# Сохраним набор нераспознанных данных в электронной таблице Excel для ручного исправления записей
water_sources_na.to_excel('Bratsk_inppw_na.xls')

C:\Users\obsid\AppData\Local\Temp/ipykernel_6636/1527049338.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  water_sources_na.to_excel('Bratsk_inppw_na.xls')


На рисунке 7 представлена таблица Excel с данными нераспознанных ВИ.

![Таблица Excel с нераспознанными адресами ВИ](src/inppwTable_na.jpg)

Рисунок 7 - Таблица Excel с нераспознанными адресами ВИ

Далее вручную и с использованием средств автоматизации Excel (автозамена) откорректируем адреса ВИ в соответствии с названиями улиц из базы зданий Братска. Например, запись "Мира 1-Южная" должна быть заменена на "Мира 1", а запись "Ленина 6" должна быть заменена на "Ленина проспект 6".

In [10]:
# Загрузим после корректировки набор данных вновь
water_sources_na = pd.read_excel("Bratsk_inppw_na.xls")

Повторим с набором действия п.4

In [9]:
# import nltk
# import time

In [ ]:
# cur_time = time.time() # Засекаем текущее время
# i=0
# # Минимальная необходимая точность совпадения записи
# accuracy_rate=0.3
# # Перебираем все записи адресов ВИ
# for key, ws in water_sources_na.iterrows():
#     item_time = time.time() # текущее время данной записи
    
#     address = ws['адрес']
#     coords=[]
#     try:
#         # Проверяем наличие точных совпадений
#         building = buildings.loc[address]
#         # Если не вызвана ошибка, значит точное совпадение найдено и можно указать координаты записи о здании
#         water_sources_na.loc[key,'geo_lon']=building['geo_lon']
#         water_sources_na.loc[key,'geo_lat']=building['geo_lat']
#         water_sources_na.loc[key,'need_check']=0

#         print("{} '{}', точное совпадение, затрачено {} с".format(i, address, round(time.time() - item_time, 2)))
#     except:
#         # Если точного совпадения нет, то ищем его  использованием nltk
#         minDiff = 1.0
#         minAddress = ''
#         for key_gis, rec_gis in buildings.iterrows():
#             buildings_gis_address = rec_gis['building']
#             sovp = nltk.edit_distance(address, buildings_gis_address)/len(address)   #Ищем минимальное редакторское расстояние
#             if (sovp < minDiff):      
#                 minDiff=sovp 
#                 minAddress=buildings_gis_address
#                 coords=[rec_gis['geo_lon'], rec_gis['geo_lat']]

#         # Если различия достаточно невелики, сохраняем их в капчествен наиболее вероятного варианта
#         if minDiff<accuracy_rate:
#             print("{} Было '{}', стало '{}', с минимальной разностью {}, затрачено {} с".format(i, address, minAddress, minDiff, round(time.time() - item_time, 2)))
#             water_sources_na.loc[key,'geo_lon']=coords[0]
#             water_sources_na.loc[key,'geo_lat']=coords[1]
#         else:
#             print('{} Подходящее значение не найдено: {}, затрачено {} с'.format(i, address, round(time.time() - item_time, 2)))

#         water_sources_na.loc[key,'need_check']=1

#     i+=1

# print('Время обработки: ', round(time.time() - cur_time, 2), 'c', sep='')

In [ ]:
# Объединим полученный набор с набором данных полученных ранее
water_sources = gpd.read_file('G:/QGIS/Bratsk/fire_data/ws_bratsk.gpkg')
# Сформируем новый набор географически привязанных данных на основе второго набора:
gdf_na = gpd.GeoDataFrame(
    water_sources_na, geometry=gpd.points_from_xy(water_sources_na.geo_lon, water_sources_na.geo_lat), crs='WGS 84')
# добавим новые данные в старый набор:
water_sources_full = water_sources.append(gdf_na)
# Сохраним набор данных в файл geopackage ws_bratsk.gpkg
water_sources_full.to_file('ws_bratsk.gpkg', driver='GPKG', layer='FRU-64')

Добавим полученный набор и повторим действия п.7 для новых меток ВИ.

## Настройка внешнего вида меток ВИ

На заключительном этапе следует настроить внешний вид меток ВИ с тем, чтобы они соответствовали требованиям БУПО. Для этого требуется заменить стандартное графическое представление приложения QGIS в виде точек на пиктограммы ПГ на основе изображений в формате svg. А так же настроить подписи меток (рис. 8).

![Полностью готовая карта с метками ВИ](src/inppwQGIS_3.jpg)

Рисунок 8 - Полностью готовая карта с метками ВИ

# Использование полученного набора данных о ВИ

- Макеты (планшеты водоисточников)
- Экспорт в эксель
- Экспорт в KML и добавление в картографические сервисы


## Формирование печатных форм планшетов водоисточников

Поскольку задача печати обновленных планшетов водоисточников для дальнейшего их использования при тушении пожаров в результате их перевода в цифровой вид не отменяется, следует рассмотреть возможность автоматического построения печатных форм водоисточников. Для этого хорошо подходит встроенный инструмент QGIS "Макеты". С его помощью можно легко создать представление произвольных картографических данных пригодной для дальнейшего экспорта как в графические форматы фалов, так и в формат PDF.

При помощи макетов QGIS был разработан пример печатной формы для полученного набора данных о водоисточниках Братска (рис. 9)

![Пример макета планшета водоисточников г.Братска](src/Planshet_WS_1.png)

Рисунок 9 - Пример макета планшета водоисточников г.Братска

Видно, что полученная карта выполнена в высоком качестве. Подобные карты могут быть созданы для любого фрагмента карты. В том числе может быть создан набор всех фрагментов карты для оформления полного набора карт планшета водоисточников. Более того, такие макеты автоматически обновляются при изменении данных карты, например при добавлении новых водоисточников. Кроме того, перечень атрибутов ПГ наполняется в соответствии с имеющимися на карте метками.

## Экспорт в электронные таблицы Excel

В повседневной деятельности зачастую может возникать задача анализа сведений о водоисточниках. В таких случаях работать с данными в виде картографических слоев QGIS неудобна. Гораздо удобнее было бы использовать данные о водоисточниках в виде электронных таблиц. Для этого можно экспортировать данные из слоя.

Для этого следует воспользоваться командой "Экспорт/Сохранить объекты как..." (рис. 10)

![Команда "Экспорт/Сохранить объекты как..."](src/Export_to.jpg)

Рисунок 10 - Команда "Экспорт/Сохранить объекты как..."

Далее, воспользовавшись одноименным диалоговым окном следует указать тип данных "Электронная таблица MS Office Open XML [XLSX]" (рис. 11 - 1). Так же можно выбрать поля слоя которые необходимо добавить в результирующую таблицу (рис. 11 - 2) и при необходимости добавить пространственные данные меток ВИ (рис. 11 - 3).

![Экспорт данных в формат Excel"](src/Export_to.jpg)

Рисунок 11 - Экспорт данных в формат Excel

## Экспорт в формат KML

Работа с пространственными данными в приложении QGIS удобна, однако, в некоторых случаях функционала данного приложения может оказаться недостаточно. Например, обмен данными ВИ между различными пользователями при помощи QGIS может быть не слишком удобным. Так же некоторые требуемые функции работы с картами городской застройки, такие как поиск кратчайшего маршрута следования могут вызывать у пользователей затруднения при работе в QGIS. Зачастую бывает гораздо удобнее работать с данными о ВИ в сторонних картографических сервисах (Google maps, Яндекс карты, Open street maps) или приложениях (2ГИС, ЭСУ ППВ). Для обмена пространственными данными между различными сервисами могут использовать различные форматы, но одним из наиболее распространенных форматов является KML (Keyhole Markup Language). На основе этого формата, например, реализовано любительское решение ЭСУ ППВ (Электронная Система Учета Противопожарного Водоснабжения [ https://эсуппв.рф/ ]). 

Для экспорта данных в KML следует использовать метод описанный выше (рис. 10). Однако, в данном случае следует выбрать тип файла "Keyhole Markup Language [KML]" (рис. 12).

![Экспорт данных в формат KML"](src/Export_to.jpg)

Рисунок 12 - Экспорт данных в формат KML

Экспорт в формат KML имеет ряд особенностей. Так, согласно спецификации формата [ KML Reference - https://developers.google.com/kml/documentation/kmlreference?hl=ru ] в описание маркеров следует включить поля "name" и "description". Первое поле является именем маркера. Второе его описанием. Эти поля следует добавить воспользовавшись инструментами приложения QGIS. В настройках же экспорта следует указать какие поля необходимо интерпретировать в качестве полей имени и описания (рис. 12 - 3).

Полученный файл KML можно добавить например в приложение 2ГИС при помощи команды "Открыть" для плагина "Справочники" (рис. 13). 

![Команда "Открыть" плагина Справочники приложения 2ГИС](src/2gis_settings_0.jpg)

Рисунок 13 - Команда "Открыть" плагина Справочники приложения 2ГИС

Далее следует настроить внешний вид маркеров ВИ в 2ГИС (рис. 14).

![Настройка внешнего вида маркеров ВИ в 2ГИС](src/2gis_settings.jpg)

Рисунок 14 - Настройка внешнего вида маркеров ВИ в 2ГИС

# Выводы по главе

В результате проведенной работы был оцифрован планшет водоисточников для города Братска. Полученный набор данных был оформлен в виде печатных форм приложения QGIS. Так же данные были экспортированы в форматы Excel и KML.